In [1]:
import numpy as np
import xarray as xr



In [2]:
#tt = xr.open_dataset('daily_datasets\chirps\chirps-v2.0.1981.days_p05.nc')
#download https://data.chc.ucsb.edu/products/CHIRPS-2.0/global_daily/netcdf/p05/chirps-v2.0.1981.days_p05.nc

In [2]:

xavier = xr.open_dataset(r'daily_datasets\xavier_cerrado_81_20.nc', chunks={'time': 100})
xavier_test = xavier.sel(lat=slice(-15, -13), lon=slice(-50, -48))
Data = xavier_test.pr.load()


In [43]:
##now a function that set to 0 the value if the  next day is bigger than the previous
#def consecutive(da, thresh=1):
#    count = 0
#    da = da.squeeze()
#    overall_counter = np.zeros(len(da))
#    #print(da.shape)
#    lenn = len(da) - 1
#    for i,j in enumerate(da):
#        if i != lenn:
#            count = count+1 if da[i] <= thresh else 0
#            ddry = count if  da[i+1] >= thresh else 0     
#        else:
#            count = count+1 if da[i] <= thresh else 0
#            ddry = 0
#        overall_counter[i] = ddry
#    #print('done')
#    return overall_counter
#
#daa = xr.apply_ufunc(
#        consecutive, Data,
#        input_core_dims=[['time']],  # Specify input dimensions
#        output_core_dims=[['new_time']],  # Specify output dimensions
#        kwargs={'thresh': 1},  # Additional arguments
#        dask='parallelized',  # Enable dask
#        vectorize=True, 
#        #output_dtypes=[float],
#    )
#
#daa = daa.rename({"new_time": "time"})
#daa["time"] = Data.time.values
#daa = daa.rename('CDD')

In [24]:
#now speed up with numba
from numba import float64, guvectorize

@guvectorize([(float64[:] , float64[:])], '(n) -> (n)')
def consecutive_numba(da, out):
    count = 0
    lenn = len(da) - 1
    overall_counter = np.zeros(len(da))
    for i,j in enumerate(da):
        if i != lenn:
            count = count+1 if da[i] <= 1 else 0
            ddry = count if  da[i+1] >= 1 else 0     
        else:
            count = count+1 if da[i] <= 1 else 0
            ddry = 0
        overall_counter[i] = ddry
    out[:] = overall_counter
    #print('ddry done')
    #return out

#create a function to apply the xr.apply_ufunc and to rename the output
def cdd_numba(da):
    daa = xr.apply_ufunc(
        consecutive_numba, da.chunk({'time': -1, 'lat': 20, 'lon': 20}),
        input_core_dims=[['time']],  # Specify input dimensions
        output_core_dims=[['new_time']],  # Specify output dimensions
        #kwargs={'thresh': 1},  # Additional arguments
        dask='parallelized',  # Enable dask
        #vectorize=True, 
        #output_dtypes=[float],
        output_sizes={'new_time': len(da.time)}
    )
    daa = daa.rename({"new_time": "time"})
    daa["time"] = da.time.values
    daa = daa.rename('CDD')
    return daa.load()

In [25]:
aaa = cdd_numba(xavier.pr)

C:\Users\luisg\AppData\Local\Temp\ipykernel_2592\2608764309.py:23: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  daa = xr.apply_ufunc(
